### Import all the required Libraries

In [210]:
import pandas as pd
import seaborn as sns
import numpy as np
import os

### Load all required datasets

In [211]:
import pandas as pd
import os

path = "overall/"

circuits = pd.read_csv(os.path.join(path, "circuits.csv"))
constructor_results = pd.read_csv(os.path.join(path, "constructor_results.csv"))
constructor_standings = pd.read_csv(os.path.join(path, "constructor_standings.csv"))
constructors = pd.read_csv(os.path.join(path, "constructors.csv"))
driver_standings = pd.read_csv(os.path.join(path, "driver_standings.csv"))
drivers = pd.read_csv(os.path.join(path, "drivers.csv"))
lap_times = pd.read_csv(os.path.join(path, "lap_times.csv"))
pit_stops = pd.read_csv(os.path.join(path, "pit_stops.csv"))
qualifying = pd.read_csv(os.path.join(path, "qualifying.csv"))
races = pd.read_csv(os.path.join(path, "races.csv"))
results = pd.read_csv(os.path.join(path, "results.csv"))
seasons = pd.read_csv(os.path.join(path, "seasons.csv"))
sprint_results = pd.read_csv(os.path.join(path, "sprint_results.csv"))
status = pd.read_csv(os.path.join(path, "status.csv"))

### Replace unwanted values

In [212]:
drivers = drivers.replace(r"\N", np.nan, regex=False)
races = races.replace(r"\N", np.nan, regex=False)
results = results.replace(r"\N", np.nan, regex=False)

### Convert columns wherever necessary

In [213]:
# driver table
drivers["number"] = pd.to_numeric(drivers["number"])
drivers["dob"] = pd.to_datetime(drivers["dob"], errors='coerce')


# races table
races_date_cols = ["date", "fp1_date", "fp2_date", "fp3_date", "quali_date", "sprint_date"]
races_time_cols = ["time", "fp1_time", "fp2_time", "fp3_time", "quali_time", "sprint_time"]

# Convert date columns
for col in races_date_cols:
    races[col] = pd.to_datetime(races[col], errors="coerce")

# Combine date + time into full datetime columns
for d_col, t_col in zip(races_date_cols, races_time_cols):
    new_col = d_col.replace("_date", "") + "_datetime"
    races[new_col] = pd.to_datetime(
        races[d_col].astype(str) + " " + races[t_col].astype(str),
        errors="coerce"
    )

# Drop original columns
races.drop(columns=races_time_cols, inplace=True)
races.drop(columns=races_date_cols, inplace=True)

# Rename main race datetime column
races.rename(columns={"date_datetime": "race_datetime"}, inplace=True)


# Results table
# Numeric with missing values → nullable integers
int_cols = ["number", "position", "milliseconds", "fastestLap", "rank"]
for col in int_cols:
    results[col] = pd.to_numeric(results[col], errors="coerce").astype("Int64")

# Float columns
results["fastestLapSpeed"] = pd.to_numeric(results["fastestLapSpeed"], errors="coerce")

# Convert fastest lap time to timedelta
def parse_lap_time(t):
    if pd.isna(t):
        return pd.NaT
    try:
        m, s = t.split(":")
        total_seconds = int(m) * 60 + float(s)
        return pd.to_timedelta(total_seconds, unit="s")
    except:
        return pd.NaT

results["fastestLapTime"] = results["fastestLapTime"].apply(parse_lap_time)

# Optional: convert positionText to category
results["positionText"] = results["positionText"].astype("category")

C:\Users\bhush\AppData\Local\Temp\ipykernel_7336\3205094942.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  races[new_col] = pd.to_datetime(
C:\Users\bhush\AppData\Local\Temp\ipykernel_7336\3205094942.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  races[new_col] = pd.to_datetime(
C:\Users\bhush\AppData\Local\Temp\ipykernel_7336\3205094942.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  races[new_col] = pd.to_datetime(
C:\Users\bhush\AppData\Local\Temp\ipykernel_7336\3205094942.py:17: UserWarning: Could not infer format, so each element will be parsed individ

### Convert columns wherever reqired

In [215]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27238 entries, 0 to 27237
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   resultId         27238 non-null  int64          
 1   raceId           27238 non-null  int64          
 2   driverId         27238 non-null  int64          
 3   constructorId    27238 non-null  int64          
 4   number           27232 non-null  Int64          
 5   grid             27238 non-null  int64          
 6   position         16285 non-null  Int64          
 7   positionText     27238 non-null  category       
 8   positionOrder    27238 non-null  int64          
 9   points           27238 non-null  float64        
 10  laps             27238 non-null  int64          
 11  time             8021 non-null   object         
 12  milliseconds     8021 non-null   Int64          
 13  fastestLap       8709 non-null   Int64          
 14  rank             8967 